In [231]:
import numpy as np
from qiskit_aer import Aer
from qiskit.circuit.library import QFT, PhaseEstimation, UnitaryGate
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from scipy.linalg import expm
from qiskit.circuit.library.arithmetic.exact_reciprocal import ExactReciprocal

class HHL:
    def __init__(self, A, b):
        self.A = np.array(A)
        self.b = np.array(b)
        self.n = len(b)  # Dimension of the system
        self.N = int(np.log2(len(b))) #how many qubit to rapresent b
        self.U = None
        self.U_circuit = None
        self.eigenvalues = None
        self.eigenvectors = None

    def create_b_circuit(self):
        """Prepare the quantum state |b>."""
        norm_b = np.linalg.norm(self.b)
        normalized_b = self.b / norm_b

        # Create a circuit and initialize the state
        """qc = QuantumCircuit(len(self.b))
        qc.initialize(normalized_b, qc.qubits)"""

        nb = int(np.log2(len(self.b)))
        vector_circuit = QuantumCircuit(nb)
        vector_circuit.initialize(normalized_b, list(range(nb)), None)

        return vector_circuit

    def _qpe_circuit(self, A, num_phase_qubits):
        """Perform Quantum Phase Estimation to find eigenvalues."""
        num_state_qubits = self.N      # Number of qubits for the state register
        num_qubits = num_phase_qubits + num_state_qubits  # Total number of qubits
        
        # Create the quantum circuit
        qc = QuantumCircuit(num_qubits)
        
        # Create the unitary matrix and its controlled version
        self.U = expm(-1j * A * 2 * np.pi)  # Unitary matrix corresponding to e^(-i*A*2π)
        self.U_circuit = UnitaryGate(self.U)
        #matrix_circuit = NumPyMatrix(matrix, evolution_time=2 * np.pi)
        
        # Add Phase Estimation
        qpe = PhaseEstimation(num_phase_qubits, self.U_circuit)
        
        # Apply the PhaseEstimation circuit to the combined registers
        qc.append(qpe, range(num_qubits))
        
        return qc


    def _inverse_eigenvalue_circuit(self):

        # check if the matrix can calculate the condition number and store the upper bound
        if (
            hasattr(self.U_circuit, "condition_bounds")
            and self.U_circuit.condition_bounds() is not None
        ):
            kappa = self.U_circuit.condition_bounds()[1]
        else:
            kappa = 1
        # Update the number of qubits required to represent the eigenvalues
        nl = max(self.N + 1, int(np.ceil(np.log2(kappa + 1))))

        # check if the matrix can calculate bounds for the eigenvalues
        if (
            hasattr(self.U_circuit, "eigs_bounds")
            and self.U_circuit.eigs_bounds() is not None
        ):
            lambda_min, lambda_max = self.U_circuit.eigs_bounds()
            # Constant so that the minimum eigenvalue is represented exactly, since it contributes
            # the most to the solution of the system. -1 to take into account the sign qubit
            #delta = self._get_delta(nl - neg_vals, lambda_min, lambda_max)
            delta = self._get_delta(nl, lambda_min, lambda_max)
            # Update evolution time
            self.U_circuit.evolution_time = (
                2 * np.pi * delta / lambda_min
            )
            # Update the scaling of the solution
            self.scaling = lambda_min
        else:
            delta = 1 / (2**nl)
            print("The solution will be calculated up to a scaling factor.")

        reciprocal_circuit = ExactReciprocal(nl, delta)#, neg_vals=neg_vals)

        return reciprocal_circuit



    def solve(self):
        """Solve the linear system using the HHL algorithm."""
        qc = QuantumCircuit(self.N + 3)
        
        # Prepare |b> state
        b_circuit = self.create_b_circuit()
        qc.compose(b_circuit, qubits=range(self.N), inplace=True)

        # Perform Quantum Phase Estimation
        qpe_circuit = self._qpe_circuit(self.A, num_phase_qubits=3)
        qc.compose(qpe_circuit,qubits=range(3+self.N), inplace=True)

        # Apply the inverse of eigenvalues
        inverse_circuit = self._inverse_eigenvalue_circuit()
        qc.compose(inverse_circuit, inplace=True)

        # Uncompute the QPE (inverse QPE)
        qc.compose(qpe_circuit.inverse(), inplace=True)

        # Measure or extract the solution state
        return qc




In [232]:
# Example usage
A = [[1,0,0,0],
     [0,1,0,0],
     [0,0,1,0],
     [0,0,0,1]]


b = [1, 0,0,0]

hhl = HHL(A, b)
solution_circuit = hhl.solve()

print(solution_circuit)

The solution will be calculated up to a scaling factor.
     ┌──────────────────────┐┌──────┐┌──────┐┌─────────┐
q_0: ┤0                     ├┤0     ├┤0     ├┤0        ├
     │  Initialize(1,0,0,0) ││      ││      ││         │
q_1: ┤1                     ├┤1     ├┤1     ├┤1        ├
     └──────────────────────┘│      ││  1/x ││         │
q_2: ────────────────────────┤2 QPE ├┤2     ├┤2 QPE_dg ├
                             │      ││      ││         │
q_3: ────────────────────────┤3     ├┤3     ├┤3        ├
                             │      │└──────┘│         │
q_4: ────────────────────────┤4     ├────────┤4        ├
                             └──────┘        └─────────┘
